In [42]:
#################################################
## Importing the necessary libraries required 
## to train 
#################################################
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [43]:
##############################################################################################################################
## A typical training procedure for a neural network is as follows:
## 
## 1> Define the neural network that has some learnable parameters (or weights)
## 2> Iterate over a dataset of inputs
## 3> Process input through the network
## 4> Compute the loss (how far is the output from being correct)
## 5> Propagate gradients back into the network’s parameters
## 6> Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient
###############################################################################################################################

###################################
## Defining the Neural Network
###################################
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()                        # Input Size         # Kernel Size         # Output Size       # Receptive Field
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)        #  28x28x1           # 3x3x1               # 28x28x32          # 3x3
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)       #  28x28x32          # 3x3x32              # 28x28x64          # 5x5
        self.pool1 = nn.MaxPool2d(2, 2)                    #  28x28x64          # MAXPOOL             # 14x14x64          # 10x10
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)      #  14x14x64          # 3x3x64              # 14x14x128         # 12x12
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)     #  14x14x128         # 3x3x128             # 14x14x256         # 14x14
        self.pool2 = nn.MaxPool2d(2, 2)                    #  14x14x256         # MAXPOOL             # 7x7x256           # 28x28
        self.conv5 = nn.Conv2d(256, 512, 3)                #  7x7x256           # 3x3x256             # 5x5x512           # 30x30
        self.conv6 = nn.Conv2d(512, 1024, 3)               #  5x5x512           # 3x3x512             # 3x3x1024          # 32x32
        self.conv7 = nn.Conv2d(1024, 10, 3)                #  3x3x1024          # 3x3x1024            # 1x1x10            # 34x34

    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        x = F.relu(self.conv7(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [44]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [45]:
###############################################
## Creating the training and the test dataset.
###############################################
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([                                          # Normalizing and transforming the images
                        transforms.ToTensor(),                                              # to tensors.
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [46]:
################################################################
## Defining the model with training and testing parameters.
################################################################
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [47]:
#########################################################
### Training the model, with provided hyperparameters ###
#########################################################
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 2):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.

loss=1.9772820472717285 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 22.29it/s]



Test set: Average loss: 1.8797, Accuracy: 2927/10000 (29%)

